In [ ]:
import sparql
from tabulate import tabulate

In [ ]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"

In [1]:
# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = """ 
prefix dc: <http://purl.org/dc/elements/1.1/> 
prefix dcterms: <http://purl.org/dc/terms/> 
prefix foaf: <http://xmlns.com/foaf/0.1/> 
prefix geo: <http://www.opengis.net/ont/geosparql#> 
prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#> 
prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rank: <http://resource.geosciml.org/ontology/timescale/rank/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix samfl: <http://def.seegrid.csiro.au/ontology/om/sam-lite#> 
prefix sf: <http://www.opengis.net/ont/sf#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix sosa: <http://www.w3.org/ns/sosa/> 
prefix thors: <http://resource.geosciml.org/ontology/timescale/thors#> 
prefix time: <http://www.w3.org/2006/time#> 
prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
prefix vann: <http://purl.org/vocab/vann/> 
prefix void: <http://rdfs.org/ns/void#> 
prefix xkos: <http://rdf-vocabulary.ddialliance.org/xkos#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
prefix tsna: <http://deeptimekb.org/tsna#> 
prefix tssc: <http://deeptimekb.org/tssc#> 
prefix tsnc: <http://deeptimekb.org/tsnc#> 
prefix tsnz: <http://deeptimekb.org/tsnz#> 
prefix tswe: <http://deeptimekb.org/tswe#> 
prefix tsbr: <http://deeptimekb.org/tsbr#> 
prefix tsba: <http://deeptimekb.org/tsba#> 
prefix tsjp: <http://deeptimekb.org/tsjp#> 
prefix tsau: <http://deeptimekb.org/tsau#> 
        """

In [2]:
sparql_prefix1 = sparql_prefix + """
SELECT DISTINCT ?tconcept  ?lbl
WHERE
{
  GRAPH <http://deeptimekb.org/iscallnew>
 {
    ?tconcept  a  gts:Age ; 
               skos:inScheme ts:isc2012-08; 
               rdfs:label ?lbl;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ts:isc2012-08
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ts:isc2012-08
                   ] .

    FILTER (?baseNum >= 150)
    FILTER (?topNum <= 150) 
  }
}
ORDER BY ?baseNum
LIMIT 1
                  """

result = sparql.query(endpoint, sparql_prefix1)
rows = [sparql.unpack_row(row) for row in result]

In [3]:
for item in rows:
    sparql_prefix2 = sparql_prefix+ f"""
SELECT DISTINCT ?All ?cLabelEn
WHERE
{'{'}
GRAPH <http://deeptimekb.org/iscallnew>
{'{'}

{'{'} #This block is to include the Age-level concept in the resulting list 
  <{item[0]}> rdfs:label ?labelAgeC . 
  ?All rdfs:label ?labelAgeC .
{'}'}
UNION
{'{'} # This and the next block are to find the broader and broaderTransitive concepts
<{item[0]}>
dc:description
      [
         skos:broaderTransitive ?All ;
         skos:inScheme  ts:isc2012-08
      ]
{'}'}
UNION 
{'{'}
<{item[0]}>
dc:description
      [
         skos:broader  ?All ;
         skos:inScheme  ts:isc2012-08
      ]

{'}'}
  
      ?All 
      rdfs:label ?cLabelEn ;
      dc:description 
      [  
        time:hasBeginning ?baseboundary1 ;
        time:hasEnd ?topboundary1 ;
        skos:inScheme ts:isc2012-08
        ] .
      
      ?baseboundary1 time:inTemporalPosition ?basetime1 .
      ?topboundary1 time:inTemporalPosition ?toptime1 .
      
      ?basetime1 dc:description
      [
        time:numericPosition ?baseNum1;
        skos:inScheme ts:isc2012-08
        ]  .
      
      ?toptime1 dc:description
      [
        time:numericPosition ?topNum1;
        skos:inScheme ts:isc2012-08
        ]  .
      
    {'}'}
{'}'}

ORDER BY (?baseNum1 - ?topNum1)
"""

In [4]:
    result = sparql.query(endpoint, sparql_prefix2)
    rows = [sparql.unpack_row(row) for row in result]
    if rows:
        print(tabulate(rows, result.variables))

All                                                                cLabelEn
-----------------------------------------------------------------  --------------------
http://resource.geosciml.org/classifier/ics/ischart/Tithonian      Tithonian Age
http://resource.geosciml.org/classifier/ics/ischart/UpperJurassic  Upper Jurassic Epoch
http://resource.geosciml.org/classifier/ics/ischart/Jurassic       Jurassic Period
http://resource.geosciml.org/classifier/ics/ischart/Mesozoic       Mesozoic Era
http://resource.geosciml.org/classifier/ics/ischart/Phanerozoic    Phanerozoic Eon


In [5]:
ConceptLevels=("Age", "Epoch", "Sub-Period", "Period", "Era", "Eon", "Super-Eon")
print("In ts:isc2012-08")
for concept in ConceptLevels:
    sparql_prefix3 = sparql_prefix + f"""
SELECT DISTINCT ?tconcept  str(?lbl) AS ?cLabel
WHERE
{'{'}
    {'{'}
       ?tconcept  a gts:GeochronologicEra ,
                 gts:{concept};  
               skos:inScheme ts:isc2012-08 ; 
               rdfs:label ?lbl ;
               time:hasBeginning ?basePosition ;
               time:hasEnd ?topPosition. 
      ?basePosition time:inTemporalPosition ?baseTime .
      ?topPosition time:inTemporalPosition ?topTime .

      ?baseTime time:numericPosition ?baseNum .
      ?topTime time:numericPosition ?topNum .
    {'}'}
   UNION 
    {'{'} 
       ?tconcept  a gts:GeochronologicEra ,
                 gts:{concept};  
               skos:inScheme ts:isc2012-08; 
               rdfs:label ?lbl ;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ts:isc2012-08
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
     {'}'} 

    FILTER (?baseNum >= 150)
    FILTER (?topNum <= 150) 

{'}'}
ORDER BY ?baseNum
LIMIT 1
                      """
    result = sparql.query(endpoint, sparql_prefix3)
    rows = [sparql.unpack_row(row) for row in result]
    if rows:
        print(concept + ':', rows[0][1])
    else:
        print(concept + ':')

In ts:isc2012-08
Age: Tithonian Age
Epoch: Upper Jurassic Epoch
Sub-Period:
Period: Jurassic Period
Era: Mesozoic Era
Eon: Phanerozoic Eon
Super-Eon:
